In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

Generating PySpark Session

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Sales Analysis").getOrCreate()

Import Libraries

In [4]:
from pyspark.sql.functions import col, when, count, lag, avg, stddev
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F
from pyspark.sql.window import Window

Load Data

In [5]:
df = spark.read.csv("./BMW sales data (2010-2024).csv", header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Engine_Size_L: double (nullable = true)
 |-- Mileage_KM: integer (nullable = true)
 |-- Price_USD: integer (nullable = true)
 |-- Sales_Volume: integer (nullable = true)
 |-- Sales_Classification: string (nullable = true)



EDA

In [7]:
df.limit(5).toPandas()
df.describe().toPandas()

,summary,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,count,50000,50000,50000,50000,50000,50000,49999,50000,50000,50000,50000
1,mean,None,2017.0157,None,None,None,None,3.2471749434988695,100307.20314,75034.6009,5067.51468,None
2,stddev,None,4.324459218093149,None,None,None,None,1.0090878552174578,57941.509343524616,25998.248881722793,2856.7671252296077,None
3,min,3 Series,2010,Africa,Black,Diesel,Automatic,1.5,3,30000,100,High
4,max,i8,2024,South America,White,Petrol,Manual,5.0,199996,119998,9999,Low


In [8]:
#null count
nulls = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()
nulls

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,0,0,0,0,0,0,1,0,0,0,0


In [9]:
df.show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|         NULL|    151748|    98740|        8300|                High|
|      i8|2013|North America|   Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|
|5 Series|2022|North America|  Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|7 Series|2020|South America| Black|   Diesel|      Manual|          2.1|    122131|    49898|   

In [10]:
df.show(5)

+--------+----+-------------+-----+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region|Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+-----+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|  Red|   Petrol|      Manual|         NULL|    151748|    98740|        8300|                High|
|      i8|2013|North America|  Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|
|5 Series|2022|North America| Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|
|      X3|2024|  Middle East| Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|7 Series|2020|South America|Black|   Diesel|      Manual|          2.1|    122131|    49898|        308

In [11]:
df.select("Model", "Year").show()

+--------+----+
|   Model|Year|
+--------+----+
|5 Series|2016|
|      i8|2013|
|5 Series|2022|
|      X3|2024|
|7 Series|2020|
|5 Series|2017|
|      i8|2022|
|      M5|2014|
|      X3|2016|
|      i8|2019|
|3 Series|2012|
|      i8|2016|
|7 Series|2020|
|5 Series|2020|
|      X1|2017|
|      M3|2014|
|      X5|2013|
|      M5|2017|
|5 Series|2017|
|      X5|2012|
+--------+----+
only showing top 20 rows


In [12]:
df.filter(df.Year > 2020).show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2022|North America|  Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|      i8|2022|       Europe| White|   Diesel|      Manual|          1.8|    196741|    55064|        7949|                High|
|      X5|2021|South America|   Red|   Diesel|      Manual|          2.2|    184981|    47527|        6273|                 Low|
|      X3|2023|  Middle East| White| Electric|   Automatic|          4.1|    194398|    85370|   

In [13]:
df2 = df.withColumn("Vehicle_age", 2024 - col("Year"))

In [14]:
df2.show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+-----------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|Vehicle_age|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+-----------+
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|         NULL|    151748|    98740|        8300|                High|          8|
|      i8|2013|North America|   Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|         11|
|5 Series|2022|North America|  Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|          2|
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|          0|
|7 Series|202

In [15]:
df.groupBy("Region").agg(F.sum("Sales_Volume")).show()

+-------------+-----------------+
|       Region|sum(Sales_Volume)|
+-------------+-----------------+
|       Europe|         42555138|
|       Africa|         41565252|
|North America|         42402629|
|South America|         41551818|
|  Middle East|         42326620|
|         Asia|         42974277|
+-------------+-----------------+



In [16]:
df.groupBy("Region").agg(
    F.sum("Sales_Volume").alias("Total_Sales")
).toPandas()

,Region,Total_Sales
0,Europe,42555138
1,Africa,41565252
2,North America,42402629
3,South America,41551818
4,Middle East,42326620
5,Asia,42974277


✔️ SUM

In [17]:
df.groupBy("Region").agg(F.sum("Sales_Volume")).show()

+-------------+-----------------+
|       Region|sum(Sales_Volume)|
+-------------+-----------------+
|       Europe|         42555138|
|       Africa|         41565252|
|North America|         42402629|
|South America|         41551818|
|  Middle East|         42326620|
|         Asia|         42974277|
+-------------+-----------------+



✔️ AVG (mean)

In [18]:
df.groupBy("Model").agg(F.avg("Price_USD")).show()

+--------+-----------------+
|   Model|   avg(Price_USD)|
+--------+-----------------+
|      i3|74800.26808142052|
|3 Series| 75566.2339499456|
|      X6|74434.60049129075|
|      X1|75262.21903719913|
|7 Series|75570.19674239177|
|      X3|75016.61685568157|
|5 Series|75287.84407665505|
|      M5|74474.93099598035|
|      M3|74841.58871515976|
|      i8|75366.27095093357|
|      X5|74708.11678181413|
+--------+-----------------+



✔️ MAX

In [19]:
df.groupBy("Fuel_Type").agg(F.max("Mileage_KM")).show()


+---------+---------------+
|Fuel_Type|max(Mileage_KM)|
+---------+---------------+
|   Diesel|         199995|
|   Hybrid|         199996|
| Electric|         199991|
|   Petrol|         199987|
+---------+---------------+



✔️ MIN

In [20]:
df.groupBy("Fuel_Type").agg(F.min("Mileage_KM")).show()

+---------+---------------+
|Fuel_Type|min(Mileage_KM)|
+---------+---------------+
|   Diesel|             42|
|   Hybrid|             23|
| Electric|             48|
|   Petrol|              3|
+---------+---------------+



✔️ COUNT

In [21]:
df.groupBy("Transmission").agg(F.count("*")).show()

+------------+--------+
|Transmission|count(1)|
+------------+--------+
|   Automatic|   24846|
|      Manual|   25154|
+------------+--------+



In [22]:
df.groupby("Region").agg(F.countDistinct("Model").alias("Total distinct model")).toPandas()

,Region,Total distinct model
0,Europe,11
1,Africa,11
2,North America,11
3,South America,11
4,Middle East,11
5,Asia,11


Multiple Aggregations at Once

In [23]:
df.groupBy("Region").agg(
    F.sum("Sales_Volume").alias("Total_Sales"),
    F.avg("Price_USD").alias("Avg_Price"),
    F.countDistinct("Model").alias("Unique_Models")
).toPandas()

,Region,Total_Sales,Avg_Price,Unique_Models
0,Europe,42555138,74988.356851,11
1,Africa,41565252,74885.771598,11
2,North America,42402629,75070.054709,11
3,South America,41551818,74973.598837,11
4,Middle East,42326620,74726.788487,11
5,Asia,42974277,75554.925006,11


groupBy on multiple columns

In [24]:
df.groupBy("Region","Fuel_type").agg(
    F.sum("Sales_Volume").alias("Total_Sales")
).toPandas()

,Region,Fuel_type,Total_Sales
0,Europe,Hybrid,10825662
1,South America,Hybrid,10362003
2,North America,Hybrid,10808682
3,South America,Petrol,10304421
4,Asia,Hybrid,11422396
5,Africa,Hybrid,10486034
6,North America,Petrol,10472845
7,Asia,Petrol,10462998
8,Europe,Diesel,10406133
9,Europe,Electric,10590064


PIVOT (Very Powerful)

In [25]:
df.groupBy("Region").pivot("Fuel_Type").agg(
    F.sum("Sales_Volume")
).toPandas()

,Region,Diesel,Electric,Hybrid,Petrol
0,Europe,10406133,10590064,10825662,10733279
1,Africa,10211670,10329085,10486034,10538463
2,North America,10259904,10861198,10808682,10472845
3,South America,10500121,10385273,10362003,10304421
4,Middle East,10491957,10395195,10627320,10812148
5,Asia,10492033,10596850,11422396,10462998


ORDER Results

Sort results by total sales:

In [26]:
df.groupBy("Region").agg(
    F.sum("Sales_Volume").alias("Total_Sales")
).orderBy(F.desc("Total_Sales")).toPandas()

,Region,Total_Sales
0,Asia,42974277
1,Europe,42555138
2,North America,42402629
3,Middle East,42326620
4,Africa,41565252
5,South America,41551818


Filtering Before Aggregation

In [27]:
df.filter(df.Year > 2020) \
  .groupBy("Region") \
  .agg(F.sum("Sales_Volume").alias("Sales_After_2020")) \
  .show()

+-------------+----------------+
|       Region|Sales_After_2020|
+-------------+----------------+
|       Europe|        11754801|
|       Africa|        11257467|
|North America|        11519426|
|South America|        11018071|
|  Middle East|        11594599|
|         Asia|        11457756|
+-------------+----------------+



Filtering using SQL Query

First Create temp view and pass table name ->

In [28]:
df.createOrReplaceTempView("mytable")

In [29]:
spark.sql("""
    SELECT *
    FROM mytable
    WHERE Year > 2023
""").toPandas()


,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
1,i3,2024,Middle East,Red,Petrol,Automatic,3.5,135958,69578,173,Low
2,X5,2024,South America,Black,Hybrid,Manual,3.6,111661,82675,2155,Low
3,5 Series,2024,Asia,Blue,Electric,Manual,3.7,157789,86855,648,Low
4,X1,2024,Europe,Red,Petrol,Automatic,2.7,24141,84346,4289,Low
...,...,...,...,...,...,...,...,...,...,...,...
3422,X3,2024,South America,Blue,Electric,Automatic,2.6,11446,51398,5395,Low
3423,3 Series,2024,Europe,Silver,Hybrid,Automatic,1.9,76618,92351,4427,Low
3424,M3,2024,Africa,Blue,Electric,Automatic,2.8,86228,38003,5911,Low
3425,X3,2024,Europe,White,Petrol,Manual,2.8,44683,33558,2601,Low


In [30]:
windowSpec = Window.partitionBy("Region").orderBy("Year")
windowSpec

In [ ]:
df2 = df.withColumn("RowNum", F.row_number().over(windowSpec)) \
        .withColumn("PrevSales", F.lag("Sales_Volume").over(windowSpec)) \
        .withColumn("NextSales", F.lead("Sales_Volume").over(windowSpec)) \
        .withColumn("RunningTotal", F.sum("Sales_Volume").over(windowSpec)) \
        .withColumn("Rank", F.rank().over(windowSpec))

df2.show(6)

+-----+----+------+------+---------+------------+-------------+----------+---------+------------+--------------------+------+---------+---------+------------+----+
|Model|Year|Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|RowNum|PrevSales|NextSales|RunningTotal|Rank|
+-----+----+------+------+---------+------------+-------------+----------+---------+------------+--------------------+------+---------+---------+------------+----+
|   X6|2010|Africa|Silver|   Petrol|      Manual|          1.7|    176650|    33796|        5935|                 Low|     1|     NULL|     7607|     2855044|   1|
|   i3|2010|Africa|  Blue|   Petrol|      Manual|          3.6|     91061|    45834|        7607|                High|     2|     5935|      968|     2855044|   1|
|   i3|2010|Africa|  Blue|   Diesel|      Manual|          1.8|    120482|   101137|         968|                 Low|     3|     7607|     5236|     2855044|   1|
|   X5|2010|Afri

by default .show() -> prints 20 rows

🟦 STEP 1 — TRANSFORMATIONS IN PYSPARK

🎯 1. select() – choose columns

In [32]:
df.select("Color", "Price_USD").show()

+------+---------+
| Color|Price_USD|
+------+---------+
|   Red|    98740|
|   Red|    79219|
|  Blue|   113265|
|  Blue|    60971|
| Black|    49898|
|Silver|    42926|
| White|    55064|
| Black|   102778|
| White|   116482|
| White|    96257|
| White|   117995|
|  Blue|    74539|
|Silver|   100015|
| White|   119486|
| Black|    34078|
|Silver|    94997|
|  Blue|   114844|
|   Red|    81934|
|Silver|   111326|
|  Blue|    82677|
+------+---------+
only showing top 20 rows


🎯 2. selectExpr() – SQL inside select

In [33]:
df.selectExpr("Model", "Year", "Price_USD * 86 AS Price_INR").show()

+--------+----+---------+
|   Model|Year|Price_INR|
+--------+----+---------+
|5 Series|2016|  8491640|
|      i8|2013|  6812834|
|5 Series|2022|  9740790|
|      X3|2024|  5243506|
|7 Series|2020|  4291228|
|5 Series|2017|  3691636|
|      i8|2022|  4735504|
|      M5|2014|  8838908|
|      X3|2016| 10017452|
|      i8|2019|  8278102|
|3 Series|2012| 10147570|
|      i8|2016|  6410354|
|7 Series|2020|  8601290|
|5 Series|2020| 10275796|
|      X1|2017|  2930708|
|      M3|2014|  8169742|
|      X5|2013|  9876584|
|      M5|2017|  7046324|
|5 Series|2017|  9574036|
|      X5|2012|  7110222|
+--------+----+---------+
only showing top 20 rows


🎯 3. filter() – filter rows

In [34]:
df.filter(df["Color"] == "Red").show()

+--------+----+-------------+-----+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region|Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+-----+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|  Red|   Petrol|      Manual|         NULL|    151748|    98740|        8300|                High|
|      i8|2013|North America|  Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|
|      M5|2017|North America|  Red|   Diesel|   Automatic|          2.6|     86900|    81934|        9755|                High|
|      X3|2015|North America|  Red|   Diesel|   Automatic|          3.7|    123910|    88091|        8635|                High|
|      X5|2021|South America|  Red|   Diesel|      Manual|          2.2|    184981|    47527|        627

OR using SQL-style string:

In [35]:
df.filter("Year > 2022").show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|      X3|2023|  Middle East| White| Electric|   Automatic|          4.1|    194398|    85370|        4645|                 Low|
|      i3|2024|  Middle East|   Red|   Petrol|   Automatic|          3.5|    135958|    69578|         173|                 Low|
|      X5|2024|South America| Black|   Hybrid|      Manual|          3.6|    111661|    82675|        2155|                 Low|
|      i8|2023|       Europe|  Blue|   Diesel|   Automatic|          3.8|     78573|   118317|   

🎯 4. where() – same as filter

In [36]:
df.where(df.Region == "Asia").show()

+--------+----+------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|  Asia|   Red|   Petrol|      Manual|         NULL|    151748|    98740|        8300|                High|
|      M5|2014|  Asia| Black|   Diesel|   Automatic|          1.6|    121156|   102778|         632|                 Low|
|      X1|2017|  Asia| Black| Electric|   Automatic|          1.6|    107918|    34078|        7291|                High|
|      X5|2013|  Asia|  Blue| Electric|      Manual|          2.6|    110142|   114844|        5561|                 Low|
|      M3|2015|  Asia|   Red|   Hybrid|      Manual|          2.0|     41514|   116902|        8897|                High|
|      M5|2021|  Asia|  

🎯 5. drop() – remove columns

In [37]:
df.drop("Mileage_KM", "Color").show()

+--------+----+-------------+---------+------------+-------------+---------+------------+--------------------+
|   Model|Year|       Region|Fuel_Type|Transmission|Engine_Size_L|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+---------+------------+-------------+---------+------------+--------------------+
|5 Series|2016|         Asia|   Petrol|      Manual|         NULL|    98740|        8300|                High|
|      i8|2013|North America|   Hybrid|   Automatic|          1.6|    79219|        3428|                 Low|
|5 Series|2022|North America|   Petrol|   Automatic|          4.5|   113265|        6994|                 Low|
|      X3|2024|  Middle East|   Petrol|   Automatic|          1.7|    60971|        4047|                 Low|
|7 Series|2020|South America|   Diesel|      Manual|          2.1|    49898|        3080|                 Low|
|5 Series|2017|  Middle East|   Diesel|      Manual|          1.9|    42926|        1232|                 Low|
|

🎯 7. cast() – change data type

In [38]:
df.withColumn("Year", col("Year").cast("int")).printSchema()

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Engine_Size_L: double (nullable = true)
 |-- Mileage_KM: integer (nullable = true)
 |-- Price_USD: integer (nullable = true)
 |-- Sales_Volume: integer (nullable = true)
 |-- Sales_Classification: string (nullable = true)



🎯 8. withColumnRenamed() – rename column

In [39]:
df.withColumnRenamed("Engine_Size_L", "Engine_Liters").show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Liters|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|         NULL|    151748|    98740|        8300|                High|
|      i8|2013|North America|   Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|
|5 Series|2022|North America|  Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|7 Series|2020|South America| Black|   Diesel|      Manual|          2.1|    122131|    49898|   

🎯 9. dropDuplicates() – remove duplicate rows

In [40]:
df.dropDuplicates(["Model", "Year"]).show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|3 Series|2010|         Asia| Black|   Petrol|      Manual|          2.1|    107572|    86660|        8650|                High|
|3 Series|2011|  Middle East|   Red|   Hybrid|   Automatic|          4.0|     22280|    55612|        6698|                 Low|
|3 Series|2012|  Middle East| White| Electric|      Manual|          2.4|    101595|   117995|         500|                 Low|
|3 Series|2013|South America| Black| Electric|   Automatic|          2.3|    173100|   101189|        1476|                 Low|
|3 Series|2014|  Middle East|  Grey|   Diesel|      Manual|          4.9|    154711|    76421|   

🎯 10. distinct() – unique rows

df.select("Region").distinct().show()